# Hierarchy Metrics Analysis

**Purpose**: Comprehensive analysis of hierarchical learning using graph-based metrics.

**This notebook**:
- ✅ Analyzes graph topology (compression, connectivity, branching)
- ✅ Computes information-theoretic metrics (MI, entropy, constraint effectiveness)
- ✅ Evaluates prediction readiness (fan-out analysis)
- ✅ Tracks training dynamics (growth exponent, reusability trends)
- ✅ Provides health scoring and actionable recommendations
- ✅ Visualizes all metrics with publication-quality plots

**15 Core Metrics**:
1-3. Compression (ratios, pattern counts, effectiveness)
4-6. Connectivity (reusability, coverage, branching)
7-9. Information Theory (MI, entropy, constraint effectiveness)
10. Prediction (fan-out)
11-12. Context (alignment, diversity)
13-14. Training Dynamics (growth, reusability trends)
15. Co-occurrence Validation

**After training**: Run this notebook to evaluate hierarchy health.

## 1. Setup and Imports

In [ ]:
# Import hierarchy metrics tools
from tools.hierarchy_metrics import (
    # Main report generator
    MetricsReport,
    
    # Individual analyzers
    GraphAnalyzer,
    InformationTheoryAnalyzer,
    PredictionAnalyzer,
    
    # Configuration
    MetricsConfig,
    ThresholdConfig,
    
    # Visualization
)

from tools.hierarchy_metrics.visualization import (
    plot_compression_ratios,
    plot_pattern_counts,
    plot_reusability_distribution,
    plot_coverage_heatmap,
    plot_mutual_information,
    plot_entropy_progression,
    plot_training_dynamics,
    plot_prediction_fanout,
    plot_health_summary,
    plot_full_dashboard
)

from tools.hierarchy_metrics.export import DashboardExporter, export_for_web

import matplotlib.pyplot as plt
%matplotlib inline

print("✓ All modules imported successfully")
print("✓ Ready for hierarchy metrics analysis")

## 2. Configure Analysis

Point to the hierarchy graph database created during training.

In [ ]:
# Path to hierarchy graph database (created during training)
GRAPH_DB_PATH = './metrics/hierarchy_graph.db'

# Optional: Path to learner (for prediction analysis)
# If you have a saved learner, load it here:
# from tools import HierarchicalConceptLearner
# learner = ...  # Load your learner
learner = None  # Set to None to skip prediction analysis

# Configuration
config = MetricsConfig(
    enable_training_collection=True,
    checkpoint_interval=1000,
    compute_mutual_information=True,
    compute_pattern_diversity=True,
    diversity_sample_size=1000,
    prediction_test_size=100,
)

print(f"✓ Configuration loaded")
print(f"  Graph DB: {GRAPH_DB_PATH}")
print(f"  Prediction analysis: {'enabled' if learner else 'disabled'}")

## 3. Generate Comprehensive Report

This runs all analyzers and computes all 15 metrics.

In [ ]:
# Generate comprehensive metrics report
print("Generating comprehensive metrics report...\n")

report = MetricsReport.generate(
    graph_db_path=GRAPH_DB_PATH,
    learner=learner,
    config=config,
    verbose=True
)

print("\n✓ Report generation complete")

## 4. Display Summary

Overall health scoring and key metrics.

In [ ]:
# Print comprehensive summary
print(report.summary())

## 5. Health Dashboard

Visual overview of hierarchy health across all categories.

In [ ]:
# Plot overall health dashboard
plot_health_summary(report, figsize=(14, 10))

## 6. Compression Metrics (1-3)

Analyze how effectively the hierarchy compresses information.

In [ ]:
# Compression ratios across levels
plot_compression_ratios(report, figsize=(12, 6))

In [ ]:
# Pattern count progression
plot_pattern_counts(report, figsize=(12, 6), log_scale=True)

In [ ]:
# Detailed compression metrics
print("Compression Ratios:")
for pair, ratio in report.compression.compression_ratios.items():
    health = report.compression.health_status.get(f"compression_{pair}")
    print(f"  {pair}: {ratio:.2f}x [{health.value}]")

print("\nPattern Counts:")
for level, count in sorted(report.compression.pattern_counts.items()):
    print(f"  {level}: {count:,}")

print("\nEffective Compression Rates:")
for level, rate in report.compression.effective_compression_rates.items():
    print(f"  {level}: {rate:.2f}x")

## 7. Connectivity Metrics (4-6)

Analyze graph connectivity and pattern reusability.

In [ ]:
# Reusability for each level
for level in sorted(report.compression.pattern_counts.keys()):
    if level in report.connectivity.reusability:
        plot_reusability_distribution(report, level, figsize=(10, 6))

In [ ]:
# Coverage heatmap
plot_coverage_heatmap(report, figsize=(10, 8))

In [ ]:
# Detailed connectivity metrics
print("Reusability (Mean Parents):")
for level, stats in sorted(report.connectivity.reusability.items()):
    health = report.connectivity.health_status.get(f"orphan_{level}")
    print(f"  {level}: {stats['mean_parents']:.2f} parents, orphan rate: {stats['orphan_rate']:.1%} [{health.value}]")

print("\nCoverage:")
for pair, rate in sorted(report.connectivity.coverage.items()):
    health = report.connectivity.health_status.get(f"coverage_{pair}")
    print(f"  {pair}: {rate:.1%} [{health.value}]")

print("\nBranching Factors:")
for level, stats in sorted(report.connectivity.branching_factors.items()):
    health = report.connectivity.health_status.get(f"branching_{level}")
    print(f"  {level}: mean={stats['mean_children']:.2f}, cv={stats['cv']:.2f} [{health.value}]")

## 8. Information-Theoretic Metrics (7-9)

Analyze mutual information and constraint effectiveness.

In [ ]:
# Mutual information and constraint effectiveness
plot_mutual_information(report, figsize=(14, 6))

In [ ]:
# Entropy progression
plot_entropy_progression(report, figsize=(12, 6))

In [ ]:
# Detailed information metrics
print("Mutual Information:")
for pair, mi_value in sorted(report.information.mutual_information.items()):
    health = report.information.health_status.get(f"mi_{pair}")
    print(f"  {pair}: {mi_value:.3f} bits [{health.value}]")

print("\nConditional Entropy:")
for key, h_cond in sorted(report.information.conditional_entropy.items()):
    print(f"  {key}: {h_cond:.3f} bits")

print("\nConstraint Effectiveness (Normalized MI):")
for pair, effectiveness in sorted(report.information.constraint_effectiveness.items()):
    health = report.information.health_status.get(f"effectiveness_{pair}")
    print(f"  {pair}: {effectiveness:.1%} [{health.value}]")

print("\nEntropy Progression:")
for level, H in sorted(report.information.entropy_progression.items()):
    print(f"  {level}: {H:.2f} bits")

## 9. Training Dynamics (13-14)

Analyze how patterns grew during training.

In [ ]:
# Training dynamics plots
if report.training_dynamics:
    plot_training_dynamics(report, figsize=(16, 6))
else:
    print("No training dynamics data available (checkpoints not captured)")

In [ ]:
# Detailed training dynamics
if report.training_dynamics:
    dynamics = report.training_dynamics
    
    print("Training Dynamics Metrics:")
    print(f"  Growth exponent: {dynamics.growth_exponent:.3f}")
    print(f"  Growth R²: {dynamics.growth_r_squared:.3f}")
    print(f"  Reusability trend slope: {dynamics.reusability_trend_slope:.4f}")
    print(f"  Checkpoints captured: {len(dynamics.checkpoints)}")
    
    # Health assessment
    growth_health = dynamics.health_status.get('growth_exponent')
    trend_health = dynamics.health_status.get('reusability_trend')
    
    print(f"\n  Growth health: {growth_health.value}")
    print(f"  Reusability trend health: {trend_health.value}")
else:
    print("No training dynamics data available")

## 10. Prediction Analysis (10)

Analyze prediction fan-out (requires learner).

In [ ]:
# Prediction fan-out
if report.prediction:
    plot_prediction_fanout(report, figsize=(12, 6))
else:
    print("Prediction analysis not available (learner not provided)")

In [ ]:
# Detailed prediction metrics
if report.prediction:
    print("Prediction Fan-Out Statistics:")
    for level, stats in sorted(report.prediction.fanout_by_level.items()):
        health = report.prediction.health_status.get(f"fanout_{level}")
        print(f"  {level}: mean={stats['mean']:.1f}, std={stats['std']:.1f}, range=[{stats['min']}, {stats['max']}] [{health.value}]")
else:
    print("Prediction analysis not available")

## 11. Full Dashboard

Plot all metrics in one comprehensive view.

In [ ]:
# Generate full dashboard with all visualizations
plot_full_dashboard(report)

## 12. Export Results

Export metrics for external analysis or web dashboards.

In [ ]:
# Export to JSON
report.export_json('./metrics/metrics_report.json')
print("✓ Exported JSON report")

# Export to CSV
report.export_csv('./metrics/csv')
print("✓ Exported CSV files")

In [ ]:
# Export for web dashboards (Plotly, Parquet, etc.)
exporter = DashboardExporter(report)

# Export all formats
exporter.export_all(
    output_dir='./metrics/dashboard',
    formats=['plotly', 'json', 'csv']  # Add 'parquet' if pandas installed
)

## 13. Interpretation Guide

### Health Status Colors:
- 🟢 **EXCELLENT**: Optimal performance, no action needed
- 🟢 **GOOD**: Healthy, minor improvements possible
- 🟡 **WARNING**: Review recommended, may impact performance
- 🟠 **POOR**: Action needed, significant issues detected
- 🔴 **CRITICAL**: Immediate attention required

### Key Metrics Interpretation:

**Compression Ratio**:
- Target: ~chunk_size (e.g., 5-15x for typical configs)
- Too low (<3x): Insufficient compression, increase chunk_size
- Too high (>20x): May lose important distinctions

**Orphan Rate**:
- Target: <10%
- High orphan rate: Increase training data or adjust chunk_size

**Coverage**:
- Target: >70%
- Low coverage: Lower patterns not being reused effectively

**Constraint Effectiveness**:
- Target: 50-80%
- Low (<30%): Upper levels not constraining lower levels
- Too high (>90%): Over-deterministic, may reduce flexibility

**Growth Exponent**:
- Target: 0.5-0.7 (sublinear growth)
- Linear (1.0): No compression benefit from hierarchy
- Superlinear (>1.0): Problematic, too many unique patterns

### Common Issues and Solutions:

1. **High orphan rate + low coverage**:
   - Solution: Increase training data
   
2. **Poor compression ratios**:
   - Solution: Adjust chunk_size or add more levels
   
3. **Low constraint effectiveness**:
   - Solution: Review hierarchy architecture, consider different chunk_size per level
   
4. **Linear growth exponent**:
   - Solution: Increase chunk_size to encourage pattern reuse

## Next Steps

### 🔍 Deep Dive Analysis
- Review specific patterns with high/low frequency
- Inspect parent-child relationships in graph DB
- Compare metrics across multiple training runs

### 🎯 Optimization
Based on metrics, adjust:
- `chunk_size`: Target compression ratio ~5-15x
- `num_levels`: Ensure coverage >70% at each transition
- `training_data`: Reduce orphan rate to <10%

### 📊 Experimentation
- Train with different configs
- Use this notebook to compare results
- Track improvements in health scores

### 🌐 Dashboards
- Use exported Plotly JSON for interactive web dashboards
- Import Parquet files into data analysis tools
- Build custom visualizations with exported data